<a href="https://colab.research.google.com/github/Pradhyumn1/chat_bot/blob/main/chat_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import Dataset, DataLoader
from nltk.stem.porter import PorterStemmer
import string
import random

**Dataset**

In [7]:
# Define the intents
intents = {
    'greeting': {
        'patterns': ['Namaste', 'Hello', 'Hi'],
        'responses': ['Namaste!', 'Hello, how can I assist you?']
    },
    'goodbye': {
        'patterns': ['Alvida', 'Goodbye'],
        'responses': ['Alvida, have a great day!', 'Goodbye, take care.']
    },
    'booking': {
        'patterns': ['I want to book a room', 'Can I make a reservation?'],
        'responses': ['Certainly! Please provide your details for the reservation.']
    },
    'reservations': {
        'patterns': ['my room11','room 12', ' 12 room reservation?'],
        'responses': ['Yes ! Your confim reservation.', 'Thanks for visitng']
    },
    'cancellation': {
        'patterns': ['I want to cancel my booking', 'Can I cancel a reservation?'],
        'responses': ['Sure, please provide your reservation reference number.']
    },
    'location': {
        'patterns': ['Where is the hotel located?', 'Can you give me the address?'],
        'responses': ['The hotel is located in the heart of Mumbai at XYZ Street.']
    },
    'contacts': {
        'patterns': ['I want to contact the hotel', 'How can I reach the customer service?'],
        'responses': ['Certainly! You can reach us at customer.service@hotelindia.com.']
    }
}

**Preprocessing**

In [8]:
stemmer = PorterStemmer()

all_words = []
tags = []
xy = []

for intent, data in intents.items():
    tag = intent
    tags.append(tag)
    for pattern in data['patterns']:
        words = [stemmer.stem(word.lower()) for word in pattern.split()]
        all_words.extend(words)
        xy.append((words, tag))

all_words = sorted(set(all_words))
tags = sorted(set(tags))


**Training**

In [9]:
# Create training data
X_train = []
y_train = []

for (pattern, tag) in xy:
    bag = [1 if stemmer.stem(word) in pattern else 0 for word in all_words]
    X_train.append(bag)
    y_train.append(tags.index(tag))

X_train = np.array(X_train)
y_train = np.array(y_train)

# Save the processed data
np.save('X_train.npy', X_train)
np.save('y_train.npy', y_train)

**Modeling**

In [10]:
# Model Definition
class SimpleChatBot(nn.Module):
    def __init__(self, input_size, output_size):
        super(SimpleChatBot, self).__init__()
        self.fc = nn.Linear(input_size, output_size)

    def forward(self, x):
        return self.fc(x)

# Training
class ChatDataset(Dataset):
    def __init__(self, X_train, y_train):
        self.n_samples = len(X_train)
        self.x_data = torch.tensor(X_train, dtype=torch.float32)
        self.y_data = torch.tensor(y_train, dtype=torch.long)

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.n_samples

# Hyperparameters
input_size = len(all_words)
output_size = len(tags)
learning_rate = 0.001
num_epochs = 1000

# Instantiate the model
model = SimpleChatBot(input_size, output_size)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Create DataLoader
dataset = ChatDataset(X_train, y_train)
train_loader = DataLoader(dataset=dataset, batch_size=8, shuffle=True)

# Training loop
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Save the trained model
torch.save(model.state_dict(), 'chatbot_model.pth')


Epoch [100/1000], Loss: 1.3531
Epoch [200/1000], Loss: 0.9504
Epoch [300/1000], Loss: 0.8422
Epoch [400/1000], Loss: 0.6434
Epoch [500/1000], Loss: 0.3570
Epoch [600/1000], Loss: 0.3861
Epoch [700/1000], Loss: 0.2467
Epoch [800/1000], Loss: 0.1436
Epoch [900/1000], Loss: 0.2907
Epoch [1000/1000], Loss: 0.2559


**chat with Bot**

In [11]:
# Chat with the bot
model.load_state_dict(torch.load('chatbot_model.pth'))
model.eval()

print("Let's chat: type 'quit' to exit")
while True:
    user_input = input('You: ')
    if user_input.lower() == 'quit':
        break

    words = [stemmer.stem(word.lower()) for word in user_input.split()]
    input_vector = [1 if stemmer.stem(word) in words else 0 for word in all_words]
    input_tensor = torch.tensor(input_vector, dtype=torch.float32).unsqueeze(0)

    # Get the model prediction
    with torch.no_grad():
        output = model(input_tensor)
        _, predicted = torch.max(output, 1)
        tag = tags[predicted.item()]

    if tag in intents:
        response = random.choice(intents[tag]['responses'])
        print(f'Bot: {response}')
    else:
        print("Bot: I'm not sure how to respond to that.")


Let's chat: type 'quit' to exit
You: hello
Bot: Hello, how can I assist you?
You: hii
Bot: Hello, how can I assist you?
You: king
Bot: Hello, how can I assist you?
You: i want a room
Bot: Certainly! Please provide your details for the reservation.
You: 12 room
Bot: Thanks for visitng
You: provide address
Bot: Hello, how can I assist you?
You: where is hotel located
Bot: The hotel is located in the heart of Mumbai at XYZ Street.
You: i want to contact hotel
Bot: Certainly! You can reach us at customer.service@hotelindia.com.
You: can you cancle my reservation
Bot: Yes ! Your confim reservation.
You: nameste
Bot: Hello, how can I assist you?
You: exit
Bot: Hello, how can I assist you?
You: quit
